In [2]:
# add h4d proj root to sys.path so we can import stuff that's in h4d_main/h4d:
# Example: suppose you want to import something from <h4d_main>/h4d/h4d/submodule/:
# "import h4d.submodule" or "from h4d.submodule import foo"
import sys, subprocess
from pathlib import Path
root_path = Path(subprocess.check_output(['git','rev-parse','--show-toplevel']).strip().decode("utf-8"))

In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
from PIL import Image

# tf.enable_eager_execution()
tf.version

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<module 'tensorflow._api.v1.version' from '/home/gbiamby/anaconda3/envs/labelar_demo/lib/python3.7/site-packages/tensorflow/_api/v1/version/__init__.py'>

In [5]:
weights_path = root_path / "training/weights/ssd_mobilenet_v2_bidmugs"
tfl_path = weights_path / "tflite"
tfl_model_path = tfl_path / "detect.tflite"
label_path = tfl_path / "labelmap.txt"
print(root_path)
assert weights_path.exists() and tfl_model_path.exists() and label_path.exists()

/home/gbiamby/school/labelar-det-demo


In [8]:
print(dir(tf.lite.TFLiteConverter))
#    'from_frozen_graph','from_saved_model','from_session'
converter = tf.lite.TFLiteConverter.from_frozen_graph(
    str(tfl_path / "tflite_graph.pb")
    , input_arrays = ["image_tensor"]
     , output_arrays = ['TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' ]
    , input_shapes={"image_tensor":[1,300,300,3]}
)
converter.allow_custom_ops = True
enable_quantize_fp16 = True
if enable_quantize_fp16:
    print("float16 post-training quantization enabled.")
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
#     converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
open(tfl_path / "converted_model.tflite", "wb").write(tflite_model)


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_calibrate_quantize_model', '_grappler_config', '_has_valid_tensors', '_int8_target_required', '_is_calibration_quantize', '_is_post_training_optimize', '_is_weight_only_quantize', '_set_batch_size', '_tf_api_names', '_tf_api_names_v1', '_validate_representative_dataset', 'convert', 'from_frozen_graph', 'from_keras_model_file', 'from_saved_model', 'from_session', 'get_input_arrays']
float16 post-training quantization enabled.


4796584

In [ ]:
input_mean, input_std = 127.5, 127.5

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
floating_model = input_details[0]['dtype'] == np.float32

print("Model data type: ", input_details[0]['dtype'])
print("input dims: h,w:", height, width)

# Input:
# img_path = Path("/tmp/grace_hopper.bmp")
# assert img_path.exists()
# img = Image.open(img_path).resize((width, height))
# input_data = np.expand_dims(img, axis=0)
# if floating_model:
#     input_data = (np.float32(input_data) - input_mean) / input_std
# Test the TensorFlow Lite model on random input data.
# input_shape = input_details[0]["shape"]
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
# interpreter.set_tensor(input_details[0]["index"], input_data)
# interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor datLLa.
# Use `tensor()` in order to get a pointer to the tensor.
# tflite_results = interpreter.get_tensor(output_details[0]["index"])

# Test the TensorFlow model on random input data.
# tf_results = model(tf.constant(input_data))

